In [1]:
model_id: str = "microsoft/Phi-3-mini-4k-instruct"
config: str = "anatomy"
eval_split: str = "test"

In [2]:
from datasets import load_dataset

dataset = load_dataset("cais/mmlu", config, split=eval_split)

display(dataset)
display(dataset[0])

Dataset({
    features: ['question', 'subject', 'choices', 'answer'],
    num_rows: 135
})

{'question': 'A lesion causing compression of the facial nerve at the stylomastoid foramen will cause ipsilateral',
 'subject': 'anatomy',
 'choices': ['paralysis of the facial muscles.',
  'paralysis of the facial muscles and loss of taste.',
  'paralysis of the facial muscles, loss of taste and lacrimation.',
  'paralysis of the facial muscles, loss of taste, lacrimation and decreased salivation.'],
 'answer': 0}

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
answer_map = ["A", "B", "C", "D"]

standard_template = """Example: How many legs does a banana have?

A: 0
B: 1
C: 2
D: 3

Provide only your final letter choice with no explanations. No yapping.

Answer: A

Question: {question}

{choices}

Provide only your final letter choice with no explanations. No yapping.

Answer: """

cot_template = """Example: How many legs does a banana have?

A: 0
B: 1
C: 2
D: 3

Think step by step. Return the answer at the end of the response after a separator ####.

Answer: A banana is not an animal; therefore it has no legs. The correct answer is A. #### A

Question: {question}

{choices}

Think step by step. Return the answer at the end of the response after a separator ####.

Answer: """

cod_template = """Example: How many legs does a banana have?

A: 0
B: 1
C: 2
D: 3

Think step by step, but only keep a minimum draft for each thinking step, with 5 words at most. Return the answer at the end of the response after a separator ####.

Answer: A banana is not an animal; therefore it has no legs. The correct answer is A. #### A

Question: {question}

{choices}

Think step by step, but only keep a minimum draft for each thinking step, with 5 words at most. Return the answer at the end of the response after a separator ####.

Answer: """

def generate_user_message(sample: dict) -> str:
    return cot_template.format(
        question=sample["question"],
        choices="\n".join(
            f"{answer_map[i]}: {choice}"
            for i, choice in enumerate(sample["choices"])
        )
    )

print(generate_user_message(dataset[0]))

Example: How many legs does a banana have?

A: 0
B: 1
C: 2
D: 3

Think step by step. Return the answer at the end of the response after a separator ####.

Answer: A banana is not an animal; therefore it has no legs. The correct answer is A. #### A

Question: A lesion causing compression of the facial nerve at the stylomastoid foramen will cause ipsilateral

A: paralysis of the facial muscles.
B: paralysis of the facial muscles and loss of taste.
C: paralysis of the facial muscles, loss of taste and lacrimation.
D: paralysis of the facial muscles, loss of taste, lacrimation and decreased salivation.

Think step by step. Return the answer at the end of the response after a separator ####.

Answer: 


In [5]:
from more_itertools import unzip
from tqdm import tqdm

def generate_prompt(sample) -> str:
    user_message = generate_user_message(sample)
    messages = [{"role": "user", "content": user_message}]
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    assert isinstance(prompt, str)
    return prompt

def batch_predict(dataset, batch_size=8):
    all_predictions = dict[int, str]()
    indexed_prompts = sorted(
        enumerate(map(generate_prompt, dataset)),
        key=lambda i_prompt: len(i_prompt[1])
    )

    for i in tqdm(range(0, len(indexed_prompts), batch_size), desc="Batch"):
        batch_idxs, batch_prompts = unzip(indexed_prompts[i: i + batch_size])
        batch_idxs, batch_prompts = list(batch_idxs), list(batch_prompts)

        batch_inputs = tokenizer(batch_prompts, padding=True, return_tensors="pt")
        batch_input_length = batch_inputs["input_ids"].shape[1]

        with torch.no_grad():
            batch_outputs = model.generate(
                **batch_inputs.to(model.device),
                max_new_tokens=512,
                do_sample=True,
                temperature=0.001,
                top_p=0.999,
            )

        for j, output in zip(batch_idxs, batch_outputs):
            all_predictions[j] = tokenizer.decode(
                output[batch_input_length:], skip_special_tokens=True
            )

    return dataset.add_column(
        "prediction", [all_predictions[i] for i in range(len(dataset))]
    )

example_preds = batch_predict(dataset.select(range(4)), batch_size=2)
display(example_preds["prediction"])

Batch:   0%|          | 0/2 [00:00<?, ?it/s]

Batch: 100%|██████████| 2/2 [00:21<00:00, 10.56s/it]


["A lesion causing compression of the facial nerve at the stylomastoid foramen will affect the nerve's function, which includes controlling the muscles of facial expression, carrying taste sensations from the anterior two-thirds of the tongue, and stimulating the lacrimal glands to produce tears. Therefore, the correct answer is D. #### D",
 'A "dished face" profile is often associated with a recessive maxilla due to failure of elongation of the cranial base. The correct answer is B. #### B',
 'The structure that collects urine in the body is the bladder. The kidneys filter the blood to produce urine, which then travels through the ureters to the bladder, where it is stored until it is expelled from the body through the urethra. The correct answer is A. #### A',
 'Ectomesenchyme, also known as neural crest cells, is a population of cells that originate from the embryonic ectoderm and migrate to various parts of the body to differentiate into diverse cell types. Among the options provid

In [6]:
def postprocess_predictions(sample: dict) -> dict:
    prediction_letter = sample["prediction"].split("####")[-1].strip()
    if len(prediction_letter) > 1:
        prediction_letter = ""
    return {
        "answer_letter": answer_map[sample["answer"]],
        "prediction_letter": prediction_letter,
    }

dataset = batch_predict(dataset).map(postprocess_predictions)
display(dataset.to_pandas())

Batch:   0%|          | 0/17 [00:00<?, ?it/s]

Batch: 100%|██████████| 17/17 [06:54<00:00, 24.36s/it]


Map:   0%|          | 0/135 [00:00<?, ? examples/s]

,question,subject,choices,answer,prediction,answer_letter,prediction_letter
0,A lesion causing compression of the facial ner...,anatomy,"[paralysis of the facial muscles., paralysis o...",0,A lesion causing compression of the facial ner...,A,D
1,"A ""dished face"" profile is often associated with",anatomy,[a protruding mandible due to reactivation of ...,1,"A ""dished face"" profile is often associated wi...",B,B
2,Which of the following best describes the stru...,anatomy,"[Bladder, Kidney, Ureter, Urethra]",0,The structure that collects urine in the body ...,A,A
3,Which of the following structures is derived f...,anatomy,"[Motor neurons, Skeletal muscles, Melanocytes,...",2,"Ectomesenchyme, also known as neural crest cel...",C,C
4,Which of the following describes the cluster o...,anatomy,"[Afferent arteriole, Glomerulus, Loop of Henle...",1,The cluster of blood capillaries found in each...,B,B
...,...,...,...,...,...,...,...
130,The dorsal roots of all spinal nerves contain,anatomy,"[sensory neuronal processes., sensory and auto...",0,The dorsal roots of all spinal nerves contain ...,A,A
131,Which of the following is the master gland of ...,anatomy,"[Adrenal, Pancreas, Pineal, Pituitary]",3,The pituitary gland is often referred to as th...,D,D
132,Loss of somatic sensation over the anterior tw...,anatomy,[lingual branch of the mandibular trigeminal n...,0,Loss of somatic sensation over the anterior tw...,A,A
133,"In men, specimens for gonococcal cultures are ...",anatomy,"[Anus, Bladder, Urethra, Testicle]",2,Gonococcal cultures are typically obtained fro...,C,C


In [7]:
fmt_acc = sum(
    pred.strip() in answer_map
    for pred in dataset["prediction_letter"]
) / len(dataset)
ans_acc = sum(
    pred.strip() == ans.strip()
    for pred, ans in zip(dataset["prediction_letter"], dataset["answer_letter"])
) / len(dataset)
print(f"Format Accuracy: {fmt_acc}")
print(f"Answer Accuracy: {ans_acc}")

Format Accuracy: 0.9777777777777777
Answer Accuracy: 0.674074074074074
